In [6]:
import albumentations as A
import pandas as pd
import numpy as np
import os
import random
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from dataset.CNN_Dataset import CNN_Dataset

from models.EfficientNet import EfficientNet
from tqdm import tqdm

from sklearn.metrics import confusion_matrix, accuracy_score
from albumentations.pytorch.transforms import ToTensorV2

In [7]:
transforms = A.Compose([
                A.Resize(224,224),
                A.HorizontalFlip(p=0.5),
                A.Normalize(),
                ToTensorV2()                
            ])

# Efficient Net

In [8]:
test_order = ['b0','b1','b2','b3','b4','b5']
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

image = cv2.imread("/home/prml/Downloads/18698369_10212844348565183_8072570633462979037_n.jpg")
image = transforms(image=image)['image'] / 255.
for net_name in test_order:
    model = EfficientNet(num_class=3,model_name=net_name, pretrained=True)
    model_save_path = f"/home/prml/chanyoung/fire-detection/saved_model//aihub_training/efficient_{net_name}.pt"
    model.load_state_dict(torch.load(model_save_path))


    model.eval()
    model.to(device)        
    with torch.no_grad():
        image = image.to(device)
        output = model(image)
        
        output = torch.nn.functional.softmax(output[0], dim=0)
        confs, predicts = torch.topk(output, 1)
        predicts = predicts.to('cpu')
        print(net_name, predicts)

KeyError: 'You have to pass data to augmentations as named arguments, for example: aug(image=image)'